# Phase 4: Running BEAST

```
Parameters
-------------
save_dir: str  
    Path to directory for saving outputs in.

cache_dir: str 
    Path to directory for cached objects in.

number_of_chains: int
    Number of chains to use (repeated runs to do) when running BEAST.

seeds: list of ints
    Seeds to use when running BEAST.

partition: str
    Name of partition to use when calling `sbatch`.

beast_ram: str
    RAM to use for each run of beast see sbatch documentation.

beast_threads: int
    Threads/CPUs to use for each run of beast see https://www.beast2.org/2021/03/31/command-line-options.html.
```


In [1]:
save_dir=runs_of_pipeline/2025-02-05
cache_dir=cache
number_of_chains=4
seeds=(42 2 88 57 200 3000 49 888)
partition=None
beast_ram=32G
beast_threads=6

In [2]:
# For some reason this still has to be called so that slurm's sbatch to use the beast_pype environment.
source activate beast_pype

(/Drives/P/conda_envs/beast_pype) 


In [5]:
job_ids_file=$cache_dir/slurm_job_ids.txt
seed_index=0
if [ -e $save_dir/beast.xml ]
    then
    for rep in $(seq 1 $number_of_chains)
        do
            job_name="beast_full_sample_run_${rep}"
            job_id=$(sbatch -p $partition -c 1 --mem $beast_ram -J $job_name -o $save_dir/run-$rep.out --parsable --time=3-00:00:00\
                        --wrap="beast -seed ${seeds[$seed_index]} -prefix '$save_dir/run-$rep-' -statefile '$save_dir/run-$rep-link_state.state'  '$save_dir/beast.xml'; echo 'slurm_job_complete'")
            echo "${job_id}" >> $job_ids_file
            seed_index=$seed_index+1
        done
    else
    for i in $save_dir/*
        do 
        if [ -d $i ] && [ "$i" != "$cache_dir" ]; then
            xml_set=${i##*/}
            for rep in $(seq 1 $number_of_chains)
                do
                    job_name="beast_sample_${xml_set}_run_${rep}"
                    job_id=$(sbatch -p $partition -c $beast_threads --mem $beast_ram -J $job_name -o $i/run-$rep.out --parsable --time=7-00:00:00\
                        --wrap="beast -seed ${seeds[$seed_index]} -threads $beast_threads -prefix '$i/run-$rep-'  -statefile '$i/run-$rep-link_state.state' '$i/beast.xml'; echo 'slurm_job_complete'")
                    echo "${job_id}" >> $job_ids_file
                    seed_index=$seed_index+1
                done
        fi
    done 
fi

Submitted batch job 48543874
Submitted batch job 48543875
Submitted batch job 48543876
Submitted batch job 48543877
Submitted batch job 48543878
Submitted batch job 48543879
Submitted batch job 48543880
Submitted batch job 48543881
Submitted batch job 48543882
Submitted batch job 48543883
Submitted batch job 48543884
Submitted batch job 48543885
Submitted batch job 48543886
Submitted batch job 48543887
Submitted batch job 48543888
Submitted batch job 48543889
Submitted batch job 48543890
Submitted batch job 48543891
Submitted batch job 48543892
Submitted batch job 48543893
Submitted batch job 48543894
Submitted batch job 48543895
Submitted batch job 48543896
Submitted batch job 48543897
Submitted batch job 48543898
Submitted batch job 48543899
Submitted batch job 48543900
Submitted batch job 48543901
Submitted batch job 48543902
Submitted batch job 48543903
Submitted batch job 48543904
Submitted batch job 48543905
Submitted batch job 48543906
Submitted batch job 48543907
Submitted batc

It may take several hours (overnight) for these beast runs to finish. You can check on there progress via `squeue --me`.

**Once the BEAST runs are finished** you can run the [Phase-6-Initial-MCMC-Diagnosis.ipynb](Phase-6-Initial-MCMC-Diagnosis.ipynb) notebook.